In [30]:
#import libraries
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

In [31]:
#import data
train=pd.read_csv('train.csv',index_col='id')
test=pd.read_csv('test.csv',index_col='id')
#test_df=test.iloc[:,0:7]

test['num_orders']=-999 #default value
df=pd.concat((train,test),axis=0)

In [32]:
df.head()

,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders
id,,,,,,,,
1379560,1,55,1885,136.83,152.29,0,0,177
1466964,1,55,1993,136.83,135.83,0,0,270
1346989,1,55,2539,134.86,135.86,0,0,189
1338232,1,55,2139,339.50,437.53,0,0,54
1448490,1,55,2631,243.50,242.50,0,0,40


In [19]:
#One hot encoding
df=pd.get_dummies(df,columns=['week','center_id','meal_id'])

In [33]:
cols_at_end = ['num_orders']
df = df[[c for c in df if c not in cols_at_end] + [c for c in cols_at_end if c in df]]

In [34]:
#train data
train_processed=df.loc[df.num_orders!=-999]
#test data
columns=[column for column in df.columns if column!='num_orders']
test_processed=df.loc[df.num_orders==-999,columns]

In [35]:
train_processed.columns

Index(['week', 'center_id', 'meal_id', 'checkout_price', 'base_price',
       'emailer_for_promotion', 'homepage_featured', 'num_orders'],
      dtype='object')

In [37]:
X=train_processed.loc[:,:'homepage_featured'].values.astype('float')
y=train_processed['num_orders'].ravel()

In [38]:
test_processed=test_processed.loc[:,:'homepage_featured'].values.astype('float')

In [39]:
print(X.shape)
print(y.shape)

(456548, 7)
(456548,)


In [40]:
#train test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(365238, 7) (365238,)
(91310, 7) (91310,)


In [41]:
#Normalization
from xgboost import XGBRegressor

In [42]:
class_xgboost=XGBRegressor(random_state=0)
class_xgboost.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]}

clf=GridSearchCV(ran_model,param_grid=parameters,cv=3)

# Fit the grid search to the data
clf.fit(X_train,y_train)

grid_search.best_params_

In [47]:
y_pred_xgb=class_xgboost.predict(X_test)
y_pred_xgb=np.absolute(y_pred_xgb)



In [48]:
from sklearn.metrics import mean_squared_log_error
xgb_error=mean_squared_log_error(y_test,y_pred_xgb,multioutput='raw_values')
final_xgb_error=100*xgb_error
final_xgb_error

array([83.14712875])

In [50]:
y_pred_xgb=class_xgboost.predict(test_processed)

In [51]:
df_submission=pd.DataFrame({'id':test.index,'num_orders':y_pred_xgb})
df_submission.to_csv('xgb_reg1.csv',index=False)